# Deep dive into SST skill
There are a few features of the SST skill that I'd like to understand better:

- Negative correlation in Western Tropical Pacific that substantially goes away over longer verification periods
- Poor MSSS in CAFE-f6 in the Southern Ocean
- "Better" initialised component of ACC for CAFE-f6 than other models, but "worse" MSSS_hist

In [1]:
import xarray as xr

## Some of the points above may be addressed by comparing our forecasts to their own analysis (CAFE60v1)